In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
)
from azure.ai.ml.entities import (
    AmlCompute, Model, Environment, BuildContext, CodeConfiguration, 
    ManagedOnlineEndpoint, ManagedOnlineDeployment, OnlineRequestSettings, ProbeSettings
)
from azure.ai.ml.constants import AssetTypes

import time, sys, json, os
from IPython.display import display, JSON

In [2]:
credential = DefaultAzureCredential()

workspace_ml_client = MLClient.from_config(
    credential
)

Found the config file in: /config.json


### Register the model asset in workspace

In [3]:
model_name = "phi3mini4kgguf"

# Make the models query fail-safe
models = workspace_ml_client.models.list()
model = next((m for m in models if m.name == model_name), None)

if model is not None:
    print(f"Model {model.name} already exists.")
else:
    # Register the foundation model from local as Model asset in ml workspace
    model = Model(
        path="models",
        type=AssetTypes.CUSTOM_MODEL,
        name=model_name,
        description="phi3 - gguf format model",
    )

    workspace_ml_client.create_or_update(model)

Model phi3mini4kgguf already exists.


### Register the environment asset in workspace

In [4]:
env_name = "phi3-gguf-env"

# Make the environments query fail-safe
envs = workspace_ml_client.environments.list()
env = next((e for e in envs if e.name == env_name), None)

# Flag to indicate if the environment is modified. 
# Set it manually:: If True, old env version is used. If False, new env version is created.
is_env_earlierone = True

if env is not None and is_env_earlierone:
    print(f"Environment {env_name} already exists.")
else:
    # Create a new environment based on the curated one
    custom_env = Environment(
        name=env_name,
        description="Custom environment with additional dependencies",
        build=BuildContext(path="../env/cpu")
    )

    # Register the custom environment
    workspace_ml_client.environments.create_or_update(custom_env)

Environment phi3-gguf-env already exists.


### Create managed online endpoint

In [5]:
# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
online_endpoint_name = "phi3-gguf-ep"

In [6]:
# managed endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for phi3 gguf",
    auth_mode="key",
)

In [7]:
# managed endpoint create async call
workspace_ml_client.begin_create_or_update(endpoint).wait()

### Create managed online deployment for the endpoint

In [8]:
deployment_name = "deploy01"
model = f"{model_name}@latest"
env = f"{env_name}@latest"

In [9]:
# managed endpoint deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../onlinescoring",
        scoring_script="score.py",
    ),
    instance_type="Standard_F16s_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=120000,
    ),
    liveness_probe=ProbeSettings(
        initial_delay=600
    ),
)

In [10]:
# managed endpoint deplyment create async call
workspace_ml_client.online_deployments.begin_create_or_update(deployment=demo_deployment).wait()

Check: endpoint phi3-gguf-ep exists
Uploading onlinescoring (0.02 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20835

......................................................................

In [ ]:
# update traffic to the deployment for 100%
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### Test endpoint with sample data

In [12]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request1.json",
)

print(response)

{"output": " 1. Eiffel Tower: Visit the iconic symbol of Paris and enjoy panoramic views from its observation decks.\n2. Notre Dame Cathedral: Explore this masterpiece of French Gothic architecture, though be aware that it is currently undergoing restoration after a fire in 2019.\n3. Louvre Museum: Visit the world's largest and most visited museum to see famous works like Leonardo da Vinci's Mona Lisa and ancient artifacts from Egyptian pharaohs.\n4. Arc de Triomphe: Stand atop this monumental arch, which offers a grand perspective of Paris and is best seen by climbing its steps or taking the elevator.\n5. Montmartre district: Visit the charming neighborhood known for its bohemian past, artistic history, and the famous Sacr\u00e9-C\u0153ur Basilica located atop the hill.\n6. Champs \u00c9lys\u00e9es: Experience one of Europe's most magnificent avenues by taking a stroll or enjoying Parisian cafes along this popular tourist destination.\n7. Mus\u00e9e d'Orsay: Visit this museum housed i

In [13]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request3.json",
)

print(response)

{"output": " The capital of India is New Delhi. It serves as the seat of government and hosts the offices of various Indian federal ministries, the Supreme Court, and the President of India. While officially declared in 1912, it became the administrative center when British ruled India, but after gaining independence in 1947, the process of shifting the capital was completed in 1946 with Delhi becoming the official capital."}
